In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings('ignore')

In [97]:
web = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/yasha_ten/.wdm/drivers/chromedriver/mac64/95.0.4638.69/chromedriver] found in cache


In [98]:
web.get('https://hands.ru/catalog/?type=order')

In [123]:
from tqdm import tqdm
prices = {}
len_cat = len(web.find_element_by_class_name('page-order-classifier').find_elements_by_class_name('category-tile__icon'))
base_url = web.current_url
for c in tqdm(range(len_cat)):
    category = web.find_element_by_class_name('page-order-classifier').find_elements_by_class_name('category-tile__icon')[c]
    cat_name = web.find_elements_by_class_name('category-tile__content')[c].text.split('\n')[0]
    print(cat_name)
    category.click()
    sleep(2)
    categ = parse_category()
    prices[cat_name] = categ
    web.get(base_url)



  0%|          | 0/20 [00:00<?, ?it/s]

Гостиная


  5%|▌         | 1/20 [00:49<15:31, 49.01s/it]

Кухни


 10%|█         | 2/20 [01:48<16:31, 55.06s/it]

...

 90%|█████████ | 18/20 [10:06<00:51, 25.78s/it]

Потолок


 95%|█████████▌| 19/20 [10:40<00:28, 28.18s/it]

Проекты и сметы


100%|██████████| 20/20 [10:52<00:00, 32.63s/it]


In [105]:
from time import sleep

def parse_services():
    services = []
    if len(web.find_elements_by_class_name('service-group')) == 0:
        print("NO ITEMS")
        web.refresh()
        sleep(3)
    for group in web.find_elements_by_class_name('service-group'):
        for x in group.find_elements_by_css_selector('a'):
            text = x.text.split('\n')
            services.append(text[1] + ' ' + text[0])
    return services


## pics 

def parse_category():
    categ = {}
    url = web.current_url
    els = len(web.find_elements_by_class_name('category-object'))
    for i in range(els):
        pic = web.find_elements_by_class_name('category-object')[i]
        name = pic.text
        pic.click()
        sleep(2)
        web.execute_script("window.scrollTo(0, 600);")
        services = parse_services()
        categ[name] = services
        web.get(url)
    return categ




In [141]:
import pandas as pd
i = 0
data = pd.DataFrame({'Категория': [], 'Объект': [], 'Услуга': [], 'Стоимость': []})
for k, v in prices.items():
    for k1, v1 in v.items():
        for el in v1:
            text = el.split()
            name = ''
            price = ''
            for word in text:
                if word.isdigit() or '₽' in word:
                    price += word
                else:
                    name += word + ' '
            data.loc[i] = [k, k1, name, price]
            i += 1
#     print(k, len(v))


In [150]:
data['Объект'] = data['Объект'].apply(lambda x: x.replace('\xad', ''))
data['Категория'] = data['Категория'].apply(lambda x: x.replace('\xad', ''))
data['Услуга'] = data['Услуга'].apply(lambda x: x.replace('\xad', ''))

In [153]:
data['Стоимость'] = data['Стоимость'].apply(lambda x: x.replace(' ', ''))